In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import plotnine
from plotnine import *

In [3]:
# 하이퍼파라미터 
class cfg:
    gpu_idx = 0
    device = torch.device("cuda:{}".format(gpu_idx) if torch.cuda.is_available() else "cpu")
    top_k = 25
    seed = 42
    neg_ratio = 100
    test_size = 0.2

# 시드 고정 
def seed_everything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    
seed_everything(cfg.seed)

In [6]:
# 경로 설정
history_df = pd.read_csv(os.path.join('history_data.csv'), encoding='utf-8')
profile_df = pd.read_csv(os.path.join('profile_data.csv'), encoding='utf-8')
meta_df = pd.read_csv(os.path.join('meta_data.csv'), encoding='utf-8')

In [7]:
# 데이터 전처리(중복 제거)
# 참고 : drop_duplicates의 subset을 무엇으로 구성하냐에 따라 제거될 항목들이 다름
# ex) 'profile_id', 'album_id' : 중복된 시청이력 모두 제거 / 'profile_id', 'album_id', 'log_time' : 같은 시간에 시청한 이력만 제거 
data = history_df[['profile_id', 'log_time', 'album_id']].drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).sort_values(by=['profile_id', 'log_time']).reset_index(drop=True)
data['rating'] = 1

cfg.n_users = data.profile_id.max() + 1
cfg.n_items = data.album_id.max() + 1

In [8]:
train, valid = train_test_split(
    data, test_size=cfg.test_size, random_state=cfg.seed
)
print(f'학습 데이터 크기 : {train.shape}')
print(f'검증 데이터 크기 : {valid.shape}')

학습 데이터 크기 : (719401, 4)
검증 데이터 크기 : (179851, 4)


In [9]:
# Matrix 형태로 변환
# lil_matrix(linked_list_matrix) : 하나의 공간으로 row, column을 표현하는 방식
#   linked_list, element_list 2개의 리스트로 구성
#   linked_list : element가 있는 column의 index가 저장
#   element_list : row, column의 값이 들어있음(이중 리스트)
# 데이터를 효율적으로 저장한다는 장점 존재

train = train.to_numpy()
matrix = sparse.lil_matrix((cfg.n_users, cfg.n_items))

# Matrix 형태로 변환
# lil matrix에 p, i, r 정보를 투입
# train(numpy)는 csr_matrix 형태로 변형
'''
lil_matrix : https://matteding.github.io/2019/04/25/sparse-matrices/#coordinate-matrix
csr_matrix : https://gaussian37.github.io/math-la-sparse_matrix/
''' 
for (p, _, i, r) in tqdm(train):
    matrix[p, i] = r

train = sparse.csr_matrix(matrix)
train = train.toarray()
print('train 형태 : \n', train)

  0%|          | 0/719401 [00:00<?, ?it/s]

train 형태 : 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
# 유저 특징 정보를 추출
# index를 profile_id로 설정
# user_features : profile_id를 key로, age를 value로 하는 딕셔너리 생성

profile_df = profile_df.set_index('profile_id')
user_features = profile_df[['age']].to_dict()
print('user_id 3의 age 정보 : ', user_features['age'][3])

user_id 3의 age 정보 :  5


In [11]:
# 아이템 특징 정보 추출
# 범주형 데이터 --> 수치형 데이터 변경
# item_featuers : album_id를 key로, age를 value로 하는 딕셔너리 생성
meta_df = meta_df.set_index('album_id')
le = LabelEncoder()
meta_df['genre_mid'] = le.fit_transform(meta_df['genre_mid'])
item_features = meta_df[['genre_mid']].to_dict()
print('album_id 749의 genre_mid 정보 : ', item_features['genre_mid'][749])

album_id 749의 genre_mid 정보 :  1


In [12]:
# 추출한 특징 정보의 속성을 저장
# n_genres(genre_mid) : categorical feature
# n_continous_feats(age) : continuous feature
cfg.n_genres = meta_df['genre_mid'].nunique()
cfg.n_continous_feats = 1

In [13]:
class NeuMF(nn.Module):
    """
    Neural Matrix Factorization Model
    참고 문헌 : https://arxiv.org/abs/1708.05031
    
    예시 : model = NeuMF(cfg)
          output = model.forward(user_ids, item_ids, [feat0, feat1])
    """
    def __init__(self, cfg):
        '''
        Args:
            cfg : config 파일로 네트워크 생성에 필요한 정보를 담음
        '''
        super(NeuMF, self).__init__()
        self.n_users = cfg.n_users
        self.n_items = cfg.n_items
        self.emb_dim = cfg.emb_dim
        self.layer_dim = cfg.layer_dim
        self.n_continuous_feats = cfg.n_continous_feats
        self.n_genres = cfg.n_genres
        self.dropout = cfg.dropout
        self.build_graph()
    
    def build_graph(self):
        '''
        Neural Matrix Factorization Model 생성
        구현된 모습은 위 그림 참고
        '''
        self.user_embedding_mf = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim)  # (batch_size, n_users, emb_dim)
        self.item_embedding_mf = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim)  # (batch_size, n_items, emb_dim)
        self.user_embedding_mlp = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim) # (batch_size, n_users, emb_dim)
        self.item_embedding_mlp = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim) # (batch_size, n_items, emb_dim)
        self.genre_embedding = nn.Embedding(num_embeddings=self.n_genres, embedding_dim=self.n_genres//2) # (batch_size, n_genres, n_genres/2의 몫)
        
        self.mlp_layers = nn.Sequential(
            nn.Linear(2*self.emb_dim + self.n_genres//2 + self.n_continuous_feats, self.layer_dim),  # (batch_size, 2*emb_dim + n_genres/2의 몫 + n_continuous_feats, layer_dim)
            nn.ReLU(),
            nn.Dropout(p=self.dropout),
            nn.Linear(self.layer_dim, self.layer_dim//2),   # (batch_size, layer_dim, layer_dim/2의 몫)
            nn.ReLU(),
            nn.Dropout(p=self.dropout)
        )
        self.affine_output = nn.Linear(self.layer_dim//2 + self.emb_dim, 1)   # output layer(layer_dim/2의 몫+emb_dim, 1)
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        """
        가중치 초기화하는 함수
        nn.Embedding인 경우 평균을 0, 표준편차를 0.01로 하는 정규분포
        nn.Linear인 경우도 동일
        bias 가 None이 아닌 경우 0으로 채우기
        """
        if isinstance(module, nn.Embedding):
            normal_(module.weight.data, mean=0.0, std=0.01)
        elif isinstance(module, nn.Linear):
            normal_(module.weight.data, 0, 0.01)
            if module.bias is not None:
                module.bias.data.fill_(0.0)
    
    def forward(self, user_indices, item_indices, feats):
        """
        Args : 
            user_indices : 유저 인덱스 정보
                ex) tensor([3100, 3100, ..., 14195, 14195])
            item_indices : 아이템 인덱스 정보
                ex) tensor([50, 65, ..., 14960, 11527])
            feats : 특징 정보
        
        Returns:
            output : 유저-아이템 쌍에 대한 추천 결과
                ex) tensor([9.4966, 22.0261, ..., -19.3535, -23.0212])
        """
        user_embedding_mf = self.user_embedding_mf(user_indices)
        item_embedding_mf = self.item_embedding_mf(item_indices)
        mf_output = torch.mul(user_embedding_mf, item_embedding_mf)  # element_wise product
        
        user_embedding_mlp = self.user_embedding_mlp(user_indices)
        item_embedding_mlp = self.item_embedding_mlp(item_indices)
        genre_embedding_mlp = self.genre_embedding(feats[1])
        input_feature = torch.cat((user_embedding_mlp, item_embedding_mlp, genre_embedding_mlp, feats[0].unsqueeze(1)), dim=-1) 
        # user_embedding, item_embedding, genre_embedding, age(unsqueeze를 통해 뒤에 1차원을 붙였음) concatenate
        # torch.cat(dim=-1) 맨 뒤 차원 늘리기
        mlp_output = self.mlp_layers(input_feature)
        
        output = torch.cat([mlp_output, mf_output], dim=-1)
        output = self.affine_output(output).squeeze(-1)
        return output

In [14]:
def make_UIdataset(train, neg_ratio):
    '''
    유저별 학습에 필요한 딕셔너리 데이터 생성
    Args:
        train : 유저-아이템의 상호작용을 담은 행렬(csr_matrix 형태)
            ex) 
                array([[0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        ...,
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.]])
        neg_ratio : negative sampling 활용할 비율
            ex) 3(positive label 1개당 negative label 3개)
            
    Returns: 
        UIdataset : 유저별 학습에 필요한 정보를 담은 딕셔너리 
            ex) {'사용자 ID': [[positive 샘플, negative 샘플], ... , [1, 1, 1, ..., 0, 0]]}
                >>> UIdataset[3]
                    [array([   16,    17,    18, ...,  9586, 18991,  9442]),
                    array([5, 5, 5, ..., 5, 5, 5]),
                    array([4, 4, 4, ..., 5, 1, 1]),
                    array([1., 1., 1., ..., 0., 0., 0.])]
    '''
    UIdataset = {}
    for user_id, items_by_user in enumerate(train):
        UIdataset[user_id] = []
        # positive 샘플 계산
        pos_item_ids = np.where(items_by_user > 0.5)[0]
        num_pos_samples = len(pos_item_ids)
        
        # negative 샘플 계산(random negative sampling)
        num_neg_samples = neg_ratio * num_pos_samples
        neg_items = np.where(items_by_user < 0.5)[0]
        neg_item_ids = np.random.choice(neg_items, min(num_neg_samples, len(neg_items)), replace=False)
        UIdataset[user_id].append(np.concatenate([pos_item_ids, neg_item_ids]))
        
        # feature 추출
        features_age = []
        for item_id in np.concatenate([pos_item_ids, neg_item_ids]):
            features_age.append(user_features['age'][user_id])
        UIdataset[user_id].append(np.array(features_age))
        
        features_genre = []
        for item_id in np.concatenate([pos_item_ids, neg_item_ids]):
            features_genre.append(item_features['genre_mid'][item_id])
        UIdataset[user_id].append(np.array(features_genre))
        
        # label 저장
        pos_labels = np.ones(len(pos_item_ids))
        neg_labels = np.zeros(len(neg_item_ids))
        UIdataset[user_id].append(np.concatenate([pos_labels, neg_labels]))
    return UIdataset

In [15]:
UIdataset = make_UIdataset(train, neg_ratio=cfg.neg_ratio)

In [16]:
def make_batchdata(user_indices, batch_idx, batch_size):
    '''
    배치 데이터로 변환
    Args:
        user_indices : 전체 유저 인덱스 정보
            ex) array([ 3100, 1800, 30098, ..., 2177, 11749, 20962])
        batch_idx : 배치 인덱스(몇번째 배치인지)
            ex) 0
        batch_size : 배치 크기 
            ex) 256
            
    Returns:
        batch_user_ids : 배치내의 유저 인덱스 정보 
            ex) [22194, 22194, 22194, 22194, 22194, ...]
        batch_item_ids : 배치내의 아이템 인덱스 정보 
            ex) [36, 407, 612, 801, 1404, ...]
        batch_feat0 : 배치내의 유저-아이템 인덱스 정보에 해당하는 feature0(age) 정보 
            ex) [6, 6, 6, 6, 6, ...]
        batch_feat1 : 배치내의 유저-아이템 인덱스 정보에 해당하는 feature1(genre_mid) 정보 
            ex) [4,  4,  4, 23,  4, ...]
        batch_labels : 배치내의 유저-아이템 인덱스 정보에 해당하는 label 정보 
            ex) [1.0, 1.0, 1.0, 1.0, 1.0, ...]
    '''
    batch_user_indices = user_indices[batch_idx*batch_size : (batch_idx+1)*batch_size]
    batch_user_ids = []
    batch_item_ids = []
    batch_feat0 = []
    batch_feat1 = []
    batch_labels = []
    
    for user_id in batch_user_indices:
        item_ids = UIdataset[user_id][0]
        feat0 = UIdataset[user_id][1]
        feat1 = UIdataset[user_id][2]
        labels = UIdataset[user_id][3]
        user_ids = np.full(len(item_ids), user_id) # 길이가 item_id이고, 지정값이 user_id인 배열 생성
        batch_user_ids.extend(user_ids.tolist())
        batch_item_ids.extend(item_ids.tolist())
        batch_feat0 .extend(feat0.tolist())
        batch_feat1.extend(feat1.tolist())
        batch_labels.extend(labels.tolist())
        
    return batch_user_ids, batch_item_ids, batch_feat0, batch_feat1, batch_labels  

def update_avg(curr_avg, val, idx):
    '''
    현재 Epoch 까지의 평균값 계산
    '''
    return (curr_avg * idx + val) / (idx + 1)

In [17]:
def train_epoch(cfg, model, optimizer, criterion):
    model.train()
    curr_loss_avg = 0.0
    
    user_indices = np.arange(cfg.n_users)
    np.random.RandomState(cfg.epoch).shuffle(user_indices)
    batch_num = int(len(user_indices) / cfg.batch_size) + 1
    bar = tqdm(range(batch_num), leave=False)
    
    for step, batch_idx in enumerate(bar):
        user_ids, item_ids, feat0, feat1, labels = make_batchdata(user_indices, batch_idx, cfg.batch_size)
        # 배치 사용자 단위로 학습
        user_ids = torch.LongTensor(user_ids).to(cfg.device)
        item_ids = torch.LongTensor(item_ids).to(cfg.device)
        feat0 = torch.FloatTensor(feat0).to(cfg.device)
        feat1 = torch.LongTensor(feat1).to(cfg.device)
        labels = torch.FloatTensor(labels).to(cfg.device)
        labels = labels.view(-1, 1)
        
        # grad 초기화
        optimizer.zero_grad()
        
        # model forward
        output = model.forward(user_ids, item_ids, [feat0, feat1])
        output = output.view(-1, 1)
        loss = criterion(output, labels)
        
        # 역전파
        loss.backward()
        
        # 최적화
        optimizer.step()
        
        if torch.isnan(loss):
            print("Loss NAN. Train finish.")
            break
        
        curr_loss_avg = update_avg(curr_loss_avg, loss, step)    # (curr_loss_avg * step + loss) / (step+1)
        
        msg = f"epoch {cfg.epoch}, "
        msg += f"loss : {curr_loss_avg.item():.5f}, "
        msg += f"lr : {optimizer.param_groups[0]['lr']:.6f}"
        bar.set_description(msg)
    
    rets = {'losses' : np.around(curr_loss_avg.item(), 5)}
    return rets

In [18]:
def recallk(actual, predicted, k=25):
    '''
    Args:
        actual -> 실제로 본 상품 리스트
        pred -> 예측한 상품 리스트
        k -> 상위 몇개의 데이터를 볼지(ex : k=5 -> 상위 5개 상품만 봄)
    Returns:
        recall_k -> recall@k
    '''
    set_actual = set(actual)
    recall_k = len(set_actual & set(predicted[:k])) / min(k, len(set_actual))
    return recall_k

def unique(sequence):
    # preserves order
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def ndcgk(actual, predicted, k=25):
    set_actual = set(actual)
    idcg = sum([1.0 / np.log(i+2) for i in range(min(k, len(set_actual)))])
    dcg = 0.0
    unique_predicted = unique(predicted[:k])
    
    for i, r in enumerate(unique_predicted):
        if r in set_actual:
            dcg += 1.0 / np.log(i + 2)
    ndcg_k = dcg / idcg
    
    return ndcg_k

def evaluation(gt, pred):
    """
    label과 prediction 사이 recall, coverage, competition 평가 함수
    Args:
        gt : 데이터 프레임 형태의 정답 데이터
        pred : 데이터 프레임 형태의 예측 데이터
    Returns:
        rets : recall, ndcg, coverage, competition 결과
            ex) {'recall' : 0.123024, 'ndcg' : 0.056809, 'coverage' : 0.017455, 'score' : 0.106470}
            
    * coverage : 얼마나 다양한 아이템을 추천했는지 측정하는 지표
    """
    gt = gt.groupby('profile_id')['album_id'].unique().to_frame().reset_index()
    gt.columns = ['profile_id', 'actual_list']
    
    evaluated_data = pd.merge(pred, gt, how='left', on='profile_id')
    evaluated_data['Recall@25'] = evaluated_data.apply(lambda x: recallk(x.actual_list, x.predicted_list), axis=1)
    evaluated_data['NDCG@25'] = evaluated_data.apply(lambda x: ndcgk(x.actual_list, x.predicted_list), axis=1)
    
    recall = evaluated_data['Recall@25'].mean()
    ndcg = evaluated_data['NDCG@25'].mean()
    coverage = (evaluated_data['predicted_list'].apply(lambda x: x[:cfg.top_k]).explode().nunique()) / meta_df.index.nunique()
    
    score = 0.75 * recall + 0.25 * ndcg
    rets = {'recall' : recall, 'ndcg' : ndcg, 'coverage' : coverage,
            'score' : score}
    return rets

In [19]:
def valid_epoch(cfg, model, data, mode='valid'):
    pred_list = []
    model.eval()
    
    query_user_ids = data['profile_id'].unique() # 추론할 모든 user array 집합
    full_item_ids = np.array([c for c in range(cfg.n_items)]) # 추론할 모든 item array 집합 
    full_item_ids_feat1 = [item_features['genre_mid'][c] for c in full_item_ids]
    for user_id in query_user_ids:
        with torch.no_grad():
            user_ids = np.full(cfg.n_items, user_id)
            
            user_ids = torch.LongTensor(user_ids).to(cfg.device)
            item_ids = torch.LongTensor(full_item_ids).to(cfg.device)
            
            feat0 = np.full(cfg.n_items, user_features['age'][user_id])
            feat0 = torch.FloatTensor(feat0).to(cfg.device)
            feat1 = torch.LongTensor(full_item_ids_feat1).to(cfg.device)
            
            eval_output = model.forward(user_ids, item_ids, [feat0, feat1]).detach().cpu().numpy()
            pred_u_score = eval_output.reshape(-1)   
        
        pred_u_idx = np.argsort(pred_u_score)[::-1]
        pred_u = full_item_ids[pred_u_idx]
        pred_list.append(list(pred_u[:cfg.top_k]))
        
    pred = pd.DataFrame()
    pred['profile_id'] = query_user_ids
    pred['predicted_list'] = pred_list
    
    # 모델 성능 확인 
    if mode == 'valid':
        rets = evaluation(data, pred)
        return rets, pred
    return pred

In [20]:
# 하이퍼 파라미터 설정 
cfg.batch_size = 256
cfg.emb_dim = 256
cfg.layer_dim = 256
cfg.dropout = 0.05
cfg.epochs = 25
cfg.learning_rate = 0.0025
cfg.reg_lambda = 0
cfg.check_epoch = 1

In [21]:
# model 생성 및 optimizer, loss 함수 설정 
model = NeuMF(cfg).to(cfg.device)
optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate, weight_decay=cfg.reg_lambda)
criterion = torch.nn.BCEWithLogitsLoss(reduction='sum')

In [22]:
total_logs = defaultdict(list)
best_scores  = 0
for epoch in range(cfg.epochs+1):
    cfg.epoch = epoch
    train_results = train_epoch(cfg, model, optimizer, criterion)
    
    # cfg.check_epoch 번의 epoch 마다 성능 확인 
    if epoch % cfg.check_epoch == 0: 
        valid_results, _ = valid_epoch(cfg, model, valid)

        logs = {
            'Train Loss': train_results['losses'],
            f'Valid Recall@{cfg.top_k}': valid_results['recall'],
            f'Valid NDCG@{cfg.top_k}': valid_results['ndcg'],
            'Valid Coverage': valid_results['coverage'],
            'Valid Score': valid_results['score'],
            }

        # 검증 성능 확인 
        for key, value in logs.items():
            total_logs[key].append(value)

        if epoch == 0:
            print("Epoch", end=",")
            print(",".join(logs.keys()))

        print(f"{epoch:02d}  ", end="")
        print("  ".join([f"{v:0.6f}" for v in logs.values()]))
        
        # 가장 성능이 좋은 가중치 파일을 저장 
        if best_scores <= valid_results['score']: 
            best_scores = valid_results['score']
            torch.save(model.state_dict(), os.path.join('model(best_scores).pth'))

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch,Train Loss,Valid Recall@25,Valid NDCG@25,Valid Coverage,Valid Score
00  23109.224610  0.165780  0.127238  0.001480  0.156145


  0%|          | 0/130 [00:00<?, ?it/s]

01  9938.766600  0.215896  0.171359  0.007624  0.204761


  0%|          | 0/130 [00:00<?, ?it/s]

...

23  19.067090  0.500121  0.358938  0.354357  0.464825


  0%|          | 0/130 [00:00<?, ?it/s]

24  16.670710  0.500691  0.358896  0.354307  0.465242


  0%|          | 0/130 [00:00<?, ?it/s]

25  14.586250  0.499939  0.358704  0.354458  0.464630
